### CUSTOM MODEL

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
## fetch the data.
housing = fetch_california_housing()
# housing.data
# housing.target
print(housing.keys())
print()
print("====================================Complete Dataset======================================")
## X_train_full, X_test, y_train_full, y_test
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1,1), random_state = 42)
print(X_train_full.shape)
print(X_test.shape)
print(y_train_full.shape)
print(y_test.shape)
print()
print("====================================Dataset After Splitting to Validation Set======================================")
## X_valid, X_test, y_valid, y_test
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print()

print("=================================Standardize the Data==============================================")
## Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled.shape)
X_valid_scaled = scaler.transform(X_valid)
print(X_valid_scaled.shape)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.shape)

## defining the input shape
input_shape = X_train.shape[1:]
print(input_shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

====================================Complete Dataset======================================
(15480, 8)
(5160, 8)
(15480, 1)
(5160, 1)

====================================Dataset After Splitting to Validation Set======================================
(11610, 8)
(3870, 8)
(11610, 1)
(3870, 1)

=================================Standardize the Data==============================================
(11610, 8)
(3870, 8)
(5160, 8)
(8,)


In [3]:
## defining the huber function
## this is the loss function
def huber_fn(y_true, y_pred):
    ## define the error from y_true and y_pred, that will be the difference
    error = y_true - y_pred
    #print(error)
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn


In [4]:
## defining the X_new_scaled
X_new_scaled = X_test_scaled

In [5]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [6]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)  ## residual block is applied here
        self.block2 = ResidualBlock(2, 30)  ## residual block is applied here
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [9]:
## model development
model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)
print(score)
print(y_pred)

Epoch 1/5
363/363 [==============================] - 2s 2ms/step - loss: 7.5527
Epoch 2/5
363/363 [==============================] - 1s 2ms/step - loss: 1.8346
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 0.7143
Epoch 4/5
363/363 [==============================] - 1s 2ms/step - loss: 0.7822
Epoch 5/5
162/162 [==============================] - 0s 1ms/step
1.0954668521881104
[[0.7269003 ]
 [0.81494254]
 [4.406591  ]
 ...
 [1.4355595 ]
 [3.5840416 ]
 [4.0431294 ]]


In [11]:
model.save("my_custom_model.ckpt")

INFO:tensorflow:Assets written to: my_custom_model.ckpt\assets


INFO:tensorflow:Assets written to: my_custom_model.ckpt\assets


In [12]:
model = keras.models.load_model("my_custom_model.ckpt")

In [13]:
history = model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 2s 2ms/step - loss: 1.7621
Epoch 2/5
363/363 [==============================] - 1s 2ms/step - loss: 0.4916
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 1.1712
Epoch 4/5
363/363 [==============================] - 1s 2ms/step - loss: 0.4903
Epoch 5/5
363/363 [==============================] - 1s 2ms/step - loss: 0.4420


In [14]:
## defining the block
block1 = ResidualBlock(2, 30)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    block1, block1, block1, block1,
    ResidualBlock(2, 30),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 [==============================] - 2s 1ms/step - loss: 0.7505
Epoch 2/5
363/363 [==============================] - 1s 1ms/step - loss: 0.3952
Epoch 3/5
363/363 [==============================] - 1s 1ms/step - loss: 0.4009
Epoch 4/5
363/363 [==============================] - 1s 1ms/step - loss: 0.4082
Epoch 5/5
162/162 [==============================] - 0s 1ms/step
